In [1]:
import pandas as pd
import json
import os
import numpy as np
from pyproj import Proj
os.chdir('/Users/taopeng/Documents/GEO877')

In [ ]:
# # download allCountries.zip from GeoNames
# if not os.path.exists(os.getcwd() + "/allCountries.zip"): # download dataset if haven't already
#   urllib.request.urlretrieve("https://download.geonames.org/export/dump/allCountries.zip", "allCountries.zip")

# # extract allCountries.zip
# import zipfile
# if not os.path.exists(os.getcwd() + "/allCountries.txt"): # extract dataset if haven't already
#   # extract zip file to the temp cloud drive
#   with zipfile.ZipFile(os.getcwd() + "/allCountries.zip", 'r') as zip_ref:
#       zip_ref.extractall(os.getcwd())

# # formulate a string list with geoname column names
# # g_headers = [np.char.strip(col_name).tolist() for col_name in pd.read_csv('geoname_headers.csv',header = None).to_numpy().flatten()]
# g_headers = ['geonameid','name','asciiname','alternatenames','latitude','longitude','feature class','feature code','country code','cc2','admin1 code','admin2 code','admin3 code','admin4 code','population','elevation','dem','timezone','modification date']

# # load allCountries.txt
# df = pd.read_csv('allCountries.txt', header = len(g_headers), names = g_headers, sep = '\t')

In [ ]:
# # make a list of all the points
# lat = df.latitude.tolist()
# lon = df.longitude.tolist()

# geonames_points = []
# for i in range(len(df)):
#     geonames_points.append([lat[i],lon[i]])

In [43]:
# open json file
with open ('GeoNames/geonames_all_points.json', encoding='utf-8') as json_file:
    pts = json.load(json_file)

# append points to latitude list, longitude list
lat = []
lon = []
for i in pts:
    lat_temp = i[0]
    lon_temp = i[1]
    lat.append(lat_temp)
    lon.append(lon_temp)

# create dataframe for the coordinates
geonames = pd.DataFrame(columns=['latitude', 'longitude'])
geonames['latitude'] = lat
geonames['longitude'] = lon
geonames

,latitude,longitude
0,42.57488,1.61616
1,42.52776,1.56446
2,42.52232,1.52319
3,42.60135,1.54340
4,42.53414,1.56678
...,...,...
12325001,7.82714,-146.01458
12325002,0.51194,-101.05669
12325003,10.21681,57.72575
12325004,-14.89653,-32.08058


## transform projection

In [45]:
df = geonames
lat = df.latitude.to_numpy()
lon = df.longitude.to_numpy()

# preprocessing to transfer longitude +180 to its equivalent -180
mask = df['longitude'] == 180 # a boolean mask of where lon == 180
df.loc[mask, 'longitude'] = -180 # set 180 to -180

# formulate zone & hemisphere info
df['zone'] = ((lon // 6) + 31).astype(int)
df['southern hemisphere'] = lat < 0
df['UTMx'] = np.zeros(df.shape[0])
df['UTMy'] = np.zeros(df.shape[0])

# UTM lists
UTMx = []
UTMy = []

for i in range(df.shape[0]):
    # parameter formulation
    proj = '+ proj=utm '
    zone = '+zone = ' + str(df.iloc[i, :]['zone'])
    south = ' +south ' if df.iloc[i, :]['southern hemisphere'] else ' '
    ellp = '+ellps=WGS84'
    # myProj = Proj('+proj=utm +zone=10 +ellps=WGS84', preserve_units=False) # define crs
    myProj = Proj(proj + zone + south + ellp, preserve_units=False) # define crs
    lon_temp = df.iloc[i, :]['longitude'] # longitude of current loc
    lat_temp = df.iloc[i, :]['latitude'] # latitude of current loc
    # projection
    UTMx_temp, UTMy_temp = myProj(lon_temp, lat_temp) # convert gcs to pcs

    # append to UTM lists
    UTMx.append(UTMx_temp)
    UTMy.append(UTMy_temp)

df['UTMx'] = UTMx
df['UTMy'] = UTMy

,latitude,longitude,zone,southern hemisphere,UTMx,UTMy
0,42.57488,1.61616,31,False,0.0,0.0
1,42.52776,1.56446,31,False,0.0,0.0
2,42.52232,1.52319,31,False,0.0,0.0
3,42.60135,1.54340,31,False,0.0,0.0
4,42.53414,1.56678,31,False,0.0,0.0
...,...,...,...,...,...,...
12325001,7.82714,-146.01458,6,False,0.0,0.0
12325002,0.51194,-101.05669,14,False,0.0,0.0
12325003,10.21681,57.72575,40,False,0.0,0.0
12325004,-14.89653,-32.08058,25,True,0.0,0.0


In [ ]:
# dict = {} # empty list

# for n in range(1,61):
#     df = geonames[geonames.zone == n]
#     lat = df.latitude.to_numpy()
#     lon = df.longitude.to_numpy()

#     # preprocessing to transfer longitude +180 to its equivalent -180
#     mask = df['longitude'] == 180 # a boolean mask of where lon == 180
#     df.loc[mask, 'longitude'] = -180 # set 180 to -180

#     # formulate zone & hemisphere info
#     df['zone'] = ((lon // 6) + 31).astype(int)
#     df['southern hemisphere'] = lat < 0
#     df['UTMx'] = np.zeros(df.shape[0])
#     df['UTMy'] = np.zeros(df.shape[0])

#     # UTM lists
#     UTMx = []
#     UTMy = []

#     for i in range(df.shape[0]):
#         # parameter formulation
#         proj = '+ proj=utm '
#         zone = '+zone = ' + str(df.iloc[i, :]['zone'])
#         south = ' +south ' if df.iloc[i, :]['southern hemisphere'] else ' '
#         ellp = '+ellps=WGS84'
#         # myProj = Proj('+proj=utm +zone=10 +ellps=WGS84', preserve_units=False) # define crs
#         myProj = Proj(proj + zone + south + ellp, preserve_units=False) # define crs
#         lon_temp = df.iloc[i, :]['longitude'] # longitude of current loc
#         lat_temp = df.iloc[i, :]['latitude'] # latitude of current loc
#         # projection
#         UTMx_temp, UTMy_temp = myProj(lon_temp, lat_temp) # convert gcs to pcs

#         # append to UTM lists
#         UTMx.append(UTMx_temp)
#         UTMy.append(UTMy_temp)

#     df['UTMx'] = UTMx
#     df['UTMy'] = UTMy

#     dict["z%s" %n] = df # zone number - points


## bandwidth

In [ ]:
import math

# function to calculate bandwidth
def bandWidth(df_temp):
    df_temp
    x = df_temp.UTMx.to_numpy()
    y = df_temp.UTMy.to_numpy()

    # std of x y coordinates
    x = df_temp.UTMx.to_numpy()
    y = df_temp.UTMy.to_numpy()
    Std_x = np.std(x)
    Std_y = np.std(y)

    # calculation of band width
    s = 0.5 *(Std_x+Std_y)

    bd = s * (math.pow(len(x), -1/6)) /1000 #bandwidth in km

    return bd

In [104]:
# # wirte to csv
# file_name = 'GeoNames/utm/z{}.csv'

# for i in range(1,44):
#     z = gn["z%s" %i] 
#     z.to_csv(file_name.format(i), index=True)

# df_gn = gn['z1']
# for i in list(gn.values())[1:4]: # from daily302 to daily 331
#     df_gn = pd.concat([df_gn,i])
# df_gn
## to csv
# df_gn.to_csv(r'GeoNames/utm/all44_1.csv', index=True)